In [27]:
import pandas as pd
import sys
sys.path.append('/home/jovyan')
from connectors import read_sql_query
from tqdm import tqdm


In [29]:
for date in tqdm(pd.date_range(start='2023-11-01', end='2023-11-02')):
    print(date.strftime("%Y-%m-%d"))
    read_sql_query(f'''
    INSERT INTO sandbox.content_rc_metrics_for_dash
     with 
    toDate('{date}') as date_,
    li as (
    select 
        shipment_number, 
        brand_id, 
        b.name as brand_name, 
        product_name, --продукт
        master_category_id,
        dictGet('dict.ods__product_hub__master_category', 'name', toUInt64(master_category_id)) as master_category,
        dictGet('dict.ods__product_hub__master_category', 'parent_id', toUInt64(master_category_id)) AS parent_category_id,
        dictGet('dict.ods__product_hub__master_category', 'name', toUInt64(parent_category_id)) AS master_category_2,
        dictGet('dict.ods__product_hub__master_category', 'parent_id', toUInt64(parent_category_id)) AS master_category_3_id,
        dictGet('dict.ods__product_hub__master_category', 'name', toUInt64(master_category_3_id)) AS master_category_3,
        countIf(sku, is_replacement = 0 and quantity > 0 ) as initial_sku,
        countIf(sku, sku_status in  ('replaced', 'cancelled')) as rc_count_sku,
        countIf(sku, sku_status in ('cancelled'))  as canc_count_sku,
        countIf(sku, sku_status in ('replaced'))  as repl_count_sku,
        sumIf(quantity,  is_replacement = 0) as initial_items,
        sumIf(found_quantity,  sku_status not in  ('replaced', 'cancelled')) as final_items,
        sum(weight_added_items) as weight_added_items,
        sum(weight_deleted_items) as weight_deleted_items

        from sandbox.damage_orders_items i 
        left join ods.product_hub__product p 
            on toUInt64(i.sku) = p.product_sku
        left join ods.product_hub__brand b
            on p.brand_id = b.id 
        where dt = date_
        group by 1,2,3,4,5,6,7,8,9,10)


        select 
            toDate(shipped_at) as date,
            shipment_id,
            master_category,
            product_name,
            brand_name,
            master_category_2,
            master_category_3,
            city,
            store_name,
            express_delivery,
            retailer_name,
            brand_tenant,
            initial_sku, --изначальное кол-во скю
            rc_count_sku, --замены+отмены скю
            canc_count_sku, --отмененные скю
            repl_count_sku, --замененные скю
            initial_items, --изначальное кол-во штук
            if(type_delivery = 'scan_pay_go', initial_items, final_items) as final_items, --итоговое кол-во штук
            weight_added_items, --добавленное кол-во штук
            if(type_delivery = 'scan_pay_go', 0, weight_deleted_items) as weight_deleted_items, --удаленное кол-во штук
            sum(initial_items) over(partition by shipment_number) as initial_items_sum, --сумма всех изначальных товаров
            sum(weight_added_items) over(partition by shipment_number) as weight_added_items_sum, --сумма всех добавленных товаров в заказе
            sum(weight_deleted_items) over(partition by shipment_number) as weight_deleted_items_sum --сумма всех удаленных товаров в заказе
        from li 
        inner join (select 

                                        s.shipment_id as shipment_id, 
                                        s.shipment_number as shipment_number,
                                        shipped_at,
                                        s.store_id as store_id,
                                        store_name, 
                                        retailer_id,
                                        retailer_name,
                                        type_store_delivery, 
                                        type_delivery,
                                        dictGet('analytics.stores_dict', 'express_delivery', toUInt64(store_id)) AS express_delivery,
                                        s.city_name as city,
                                        multiIf(api_client_id = 0 and tenant_id not in ('sbermarket', 'smbusiness'), tenant_id||'_wl'
                                    , coalesce(ic.name, tenant_id)) brand_tenant
                                    from analytics.bi_shipments_financial s
                                    left join analytics.int_channels ic on ic.id = dictGet('analytics.int_api_clients_dict', 'channel_id', toUInt64(s.api_client_id))
                                    where toDate(shipped_at) = date_
                                    and shipment_state = 'shipped'
                                ) sh 
                        on li.shipment_number = sh.shipment_number


    ''')

  0%|          | 0/2 [00:00<?, ?it/s]

2023-11-01


 50%|█████     | 1/2 [00:36<00:36, 36.87s/it]

2023-11-02


100%|██████████| 2/2 [01:04<00:00, 32.33s/it]
